In [ ]:
from torch.utils.data import DataLoader

from src.concept_bottleneck.dataset import (
    CUB200ImageToAttributes,
    NUM_ATTRIBUTES,
)

batch_size = 16
num_workers = 2

training_data = CUB200ImageToAttributes(train=True)
training_dataloader = DataLoader(
    training_data, batch_size=batch_size, num_workers=num_workers, shuffle=True
)

test_data = CUB200ImageToAttributes(train=False)
test_dataloader = DataLoader(test_data, batch_size=batch_size, num_workers=num_workers)


In [ ]:
import torch

device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

model: torch.nn.Module = torch.hub.load(
    "pytorch/vision:v0.10.0",
    "inception_v3",
    init_weights=False,
    num_classes=NUM_ATTRIBUTES,
).to(device)


In [ ]:
import numpy.typing as npt
import numpy as np


def train(
    model: torch.nn.Module,
    dataloader: DataLoader[tuple[torch.Tensor, npt.NDArray[np.float32]]],
    loss_fn: torch.nn.Module,
    optimizer: torch.optim.Optimizer,
    device: str,
):
    model.train()
    size = len(dataloader.dataset)  # type: ignore
    for batch, (x, y) in enumerate(dataloader):
        x = x.to(device)
        y = y.to(device)

        logits, aux_logits = model(x)
        loss = loss_fn(logits, y) + 0.4 * loss_fn(aux_logits, y)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            print(f"loss: {loss.item():>7f} [{batch * len(x):>5d}/{size:>5d}]")


def test(
    model: torch.nn.Module,
    dataloader: DataLoader[tuple[torch.Tensor, npt.NDArray[np.float32]]],
    loss_fn: torch.nn.Module,
    device: str,
):
    model.eval()
    test_loss = 0
    correct = 0
    total_correct = 0
    with torch.no_grad():
        for x, y in dataloader:
            x = x.to(device)
            y = y.to(device)

            logits = model(x)
            test_loss += loss_fn(logits, y).item()

            correct_attributes = (
                ((torch.sigmoid(logits) >= 0.5) == (y >= 0.5)).sum().item()
            )
            correct += correct_attributes / NUM_ATTRIBUTES

            total_correct += ( # Count the number of images with all attributes correct
                torch.all((torch.sigmoid(logits) >= 0.5) == (y >= 0.5), dim=1)
                .sum()
                .item()
            )

    test_loss /= len(dataloader)
    accuracy = correct / len(dataloader.dataset)  # type: ignore
    total_accuracy = total_correct / len(dataloader.dataset)  # type: ignore
    print(f"Total accuracy: {total_accuracy:>0.10f}%")

    return test_loss, accuracy


In [ ]:
from src.concept_bottleneck.train import TrainFn, TestFn, run_epochs

loss_fn = torch.nn.MSELoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.001)

train_fn: TrainFn = lambda model: train(
    model, training_dataloader, loss_fn, optimizer, device
)
test_fn: TestFn = lambda model, dataloader: test(model, dataloader, loss_fn, device)

epochs = 100

run_epochs(
    epochs,
    model,
    train_fn,
    test_fn,
    training_dataloader,
    test_dataloader,
    save_name="image-to-attributes.pth",
)
